In [189]:
import pandas as pd
import numpy as np
import itertools
import random
import itertools

def create_imdb(): 
    df = pd.read_csv('imdb.tsv', sep='\t', nrows=1000)
    print(df['genres'].nunique())
    print(df['genres'].count())
    #print(df['genres'])
    return df

def create_queries(df,ref_keyword):
    qs = df[df['genres'].str.contains(ref_keyword,na=False)]['genres']
    #print('query count',len(list(set(qs.tolist()))))
    return list(set(qs.tolist()))

def create_candidates_multiple_predicates(df,ref_keyword,pred_num):
    preds = set(df[df['genres'].str.contains(ref_keyword,na=False)]['genres'])
    return list(itertools.combinations(preds, pred_num))
   
def query_result(df,preds): 
    if type(preds) is not tuple:
        preds = [preds]
    else:
        preds = list(preds)
    return set(df.index[df['genres'].isin(preds)].tolist()) 
            
def get_all_results(df,qs):
    res = {}
    for q in qs: 
        res[q] = query_result(df,q)
    return res
    
def utility(real_query_res, cand_res):
    if not real_query_res.issubset(cand_res):
        return 0
    return 1.0/(len(cand_res.symmetric_difference(real_query_res))+1)

def cands_utility(cands,real_query,res):
    return [utility(res[real_query],res[c]) for c in cands]

def get_valid_candidates(query,cands,all_res):
    vcands = []
    for c in cands:
        #if query in c:
        #    continue
        if all_res[str(query)].issubset(all_res[str(c)]):
            vcands.append(c)
    if len(vcands) == 0:
        print('no containing query', query)
        #print('num of containing candidates:',len(vcands))
    return vcands

def private_query(real_query,epsilon,cands,res,sensitivity):
    # get the score of all candidates
    scores = cands_utility(cands,real_query,res)
    # exponential mechanism
    # sensitivity is assumed to be 1
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(epsilon * s / (2 * sensitivity)) for s in scores]
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)
    # Choose a candidate based on the probabilities
    return cands[np.random.choice(list(range(0,len(scores))), 1, p=probabilities)[0]]

    

def main(keyword,gencont):
    df = create_imdb()
    ref_keyword = keyword #''#'Drama,Short'
    epsilon = 1 
    sensitivity = 1
    # get queries
    qs = create_queries(df,ref_keyword)
    qs = random.sample(qs, min(len(qs),20))
    # candidates with multiple predicates
    #cands = create_candidates(df, ref_keyword)
    cands = create_candidates_multiple_predicates(df,ref_keyword,2)
    print('cand count:',len(cands))
    # compute all query and candidate results
    all_res = get_all_results(df,qs).copy()
    all_res.update(get_all_results(df,cands))
    for q in qs:
        # filter candidates that do not contain query 
        if gencont:
            val_cands = get_valid_candidates(q,cands,all_res)
        else:
            val_cands = cands
        if len(val_cands) == 0:
            continue
        pq = private_query(q,epsilon,val_cands,all_res,sensitivity)
        print('query:',q)
        #print('query result:',all_res[q])
        print('private query:',pq)
        #print('private query result:',all_res[pq])
    
    
main('Drama,Short',gencont=True)


88
1000
cand count: 36


KeyError: "('Comedy,Drama,Short', 'Adventure,Drama,Short')"

In [48]:
# here, the candidates are generated by enumerating pairs of values in the domain of a query column ('Model Name')
import pandas as pd
import numpy as np
import itertools
import random
import itertools

def create_imdb(): 
    df = pd.read_csv('imdb.tsv', sep='\t', nrows=1000)
    print(df['genres'].nunique())
    print(df['genres'].count())
    print(df)
    return df

def create_ebay(): 
    df = pd.read_csv('ebay_product.csv')
    print(df['Model Name'].nunique())
    print(df['Model Name'].count())
    return df

def create_queries(df,query_column,ref_keyword):
    qs = df[df[query_column].str.contains(ref_keyword,na=False)][query_column]
    print('query count',len(list(set(qs.tolist()))))
    return list(set(qs.tolist()))

def create_candidates_multiple_predicates(df,query_column,ref_keyword,pred_num):
    preds = set(df[df[query_column].str.contains(ref_keyword,na=False)][query_column])
    return list(itertools.combinations(preds, pred_num))
   
def query_result(df,query_column,preds): 
    if type(preds) is not tuple:
        preds = [preds]
    else:
        preds = list(preds)
    return set(df.index[df[query_column].isin(preds)].tolist()) 
            
def get_all_results(df,query_column,qs):
    res = {}
    for q in qs: 
        res[q] = query_result(df,query_column,q)
    return res
    
def utility(real_query_res, cand_res):
    if not real_query_res.issubset(cand_res):
        return 0
    return 1.0/(len(cand_res.symmetric_difference(real_query_res))+1)

def cands_utility(cands,real_query,res):
    return [utility(res[real_query],res[c]) for c in cands]

def get_valid_candidates(query,cands,all_res):
    vcands = []
    for c in cands:
        #if query in c:
        #    continue
        if all_res[query].issubset(all_res[c]):
            vcands.append(c)
    if len(vcands) == 0:
        print('no containing query', query)
        #print('num of containing candidates:',len(vcands))
    return vcands

def private_query(real_query,epsilon,cands,res,sensitivity):
    # get the score of all candidates
    scores = cands_utility(cands,real_query,res)
    # exponential mechanism
    # sensitivity is assumed to be 1
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(epsilon * s / (2 * sensitivity)) for s in scores]
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)
    # Choose a candidate based on the probabilities
    return cands[np.random.choice(list(range(0,len(scores))), 1, p=probabilities)[0]]

    

def main(dataset,keyword,query_column,gencont):
    if dataset=='imdb':
        df = create_imdb()
    if dataset=='ebay':
        df = create_ebay()
    ref_keyword = keyword 
    epsilon = 1 
    sensitivity = 1
    # get queries
    qs = create_queries(df,query_column,ref_keyword)
    qs = random.sample(qs, min(len(qs),20))
    # candidates with multiple predicates
    #cands = create_candidates(df, ref_keyword)
    cands = create_candidates_multiple_predicates(df,query_column,ref_keyword,2)
    print('cand count:',len(cands))
    # compute all query and candidate results
    all_res = get_all_results(df,query_column,qs).copy()
    all_res.update(get_all_results(df,query_column,cands))
    for q in qs:
        # filter candidates that do not contain query 
        if gencont:
            val_cands = get_valid_candidates(q,cands,all_res)
        else:
            val_cands = cands
        if len(val_cands) == 0:
            continue
        pq = private_query(q,epsilon,val_cands,all_res,sensitivity)
        print('query:',q)
        #print('query result:',all_res[q])
        print('private query:',pq)
        #print('private query result:',all_res[pq])
    
# query_column: ebay: 'Model Name', imdb: 'genres' 
# gencont='False': generalizing query does not need to be containing query result 
#main(dataset='imdb',keyword='Drama,Short',query_column='genres',gencont=False)
main(dataset='ebay',keyword='apple',query_column='Model Name',gencont=True)


25204
29968
query count 57
cand count: 1596
query: Watch smartwatch marked 12. similar apple iwatch 5 Compatible with iOS and Android.
private query: ('apple iphone 7 32gb unlocked black', 'Watch smartwatch marked 12. similar apple iwatch 5 Compatible with iOS and Android.')
query: series 4 apple watch 44mm
private query: ('series 4 apple watch 44mm', 'apple watch series 5 gps cellular 44mm')
query: apple watch series 3 38mm
private query: ('apple watch series 3 38mm', 'apple watch series 3 42mm cellular')
query: Charger Dock for apple Watch Charger with Build-in 3.3 FT USB Charging Cable
private query: ('Charger Dock for apple Watch Charger with Build-in 3.3 FT USB Charging Cable', 'apple iphone 7 - 32gb - roségold')
query: NEW apple watch series 6 GPS, 40mm Space Gray Aluminum Case With Black Sport
private query: ('NEW apple watch series 6 GPS, 40mm Space Gray Aluminum Case With Black Sport', 'apple watch series 3 42mm Gps')
query: apple watch series 3 38mm rose gold gps
private quer

In [16]:
#%%capture cap --no-stderr

# here, the candidates are generated by enumerating values in the domains of all columns except the query column ('Model Name')
import pandas as pd
import numpy as np
import itertools
import random
import itertools
import json
import pickle

def create_imdb(): 
    df = pd.read_csv('imdb.tsv', sep='\t', nrows=5000)
    #df = df.fillna('nan')
    return df.apply(lambda x: x.astype(str).str.lower())
    #return df

def create_ebay(): 
    df = pd.read_csv('ebay_product.csv')
    #df = df.fillna('nan')
    return df.apply(lambda x: x.astype(str).str.lower())
    #return df

def create_queries_nosave(df,query_column,ref_keyword):
    qs = df[df[query_column].str.contains(ref_keyword,na=False)][query_column].drop_duplicates().values
    return [[q] for q in qs] 

def create_queries(dataset_name,query_num,query_column,ref_keyword):
    if dataset_name=='ebay':
        df = create_ebay()
    elif dataset_name=='imdb':
        df = create_imdb()  
    qps = df[df[query_column].str.contains(ref_keyword,na=False)][query_column].drop_duplicates().values
    qs =[[q] for q in qps]
    print('NUM OF GENERATED QUERIES: ' + str(len(qs)))
    qs = random.sample(qs, min(len(qs),query_num))
    print('NUM OF QUERIES: ' + str(len(qs)))
    with open(dataset_name+'_queries.json', 'w') as f:
        json.dump(qs, f)
  
def read_queries(dataset_name):
    with open(dataset_name+'_queries.json', 'r') as f:
        qs = json.load(f)
    return qs

def create_candidates_columns(df,pred_cols,max_preds_num):
    df2=df[pred_cols].drop_duplicates()
    df2 = df2.dropna()
    preds = list(df2.values)
    if max_preds_num is None: 
        max_preds_num = len(preds)
    if len(preds) > max_preds_num:
        preds = random.sample(preds, max_preds_num)
    return preds

# _conjunctive
def get_all_cand_results(df,pred_cols,preds,query_type):  
    df2 = df[pred_cols]
    res = {}
    for q in preds:
        q = np.array(list(q))
        # making the query a readable string
        predstr = [str(z)+' = '+str(w) for (z,w) in zip(pred_cols,q)]
        qstr = query_type.join(predstr) 
        #
        #res[str(q)] = set(df[(df2 == q).all(1)].index)
        res[qstr] = set(df[(df2 == q).all(1)].index)
    return res

# _disjunctive
def get_all_cand_results_disjunctive(df,pred_cols,preds,query_type):  
    dfs = {pc:df[[pc]] for pc in pred_cols}
    res = {}
    for p in preds:
        # making the query a readable string
        predstr = [str(z)+' = '+str(w) for (z,w) in zip(pred_cols,p)]
        qstr = query_type.join(predstr) 
        #
        res[qstr] = set()
        for (pc,q) in zip(pred_cols,p):
            df2 = dfs[pc]
            q = np.array(list([q]))
            res[qstr] = res[qstr].union(set(df[(df2 == q).all(1)].index))  
    return res
    
    
def utility1(real_query_res, cand_res):
    return len(cand_res.intersection(real_query_res))/len(cand_res)
    
def utility2(real_query_res, cand_res):
    return len(cand_res.intersection(real_query_res))*(min(len(cand_res),len(real_query_res))/(max(len(cand_res),len(real_query_res))+1))
    
def utility3(real_query_res, cand_res):
    if real_query_res.issubset(cand_res): 
        return 1.0/(len(cand_res.difference(real_query_res))+1)
    else: 
        return 0
    return len(cand_res.intersection(real_query_res))/(len(cand_res.symmetric_difference(real_query_res))+1)

def cands_utility(cands,real_query,res,utility_fn):
    return [utility_fn(res[real_query],res[c]) for c in cands]

def private_query(real_query,epsilon,cands,res,sensitivity,utility_fn):
    # get the score of all candidates
    scores_all = cands_utility(cands,real_query,res,utility_fn)
    cands_small = []
    scores = []
    for i in range(len(scores_all)):  
        if scores_all[i] != 0:
            scores.append(scores_all[i])
            cands_small.append(cands[i])
        elif len(scores)<200:
            scores.append(scores_all[i])
            cands_small.append(cands[i])
    cands = cands_small.copy()
    #print('NUM OF CONSIDERED CANDIDATES: %d' %(len(cands)))
    # exponential mechanism
    # sensitivity is assumed to be 1
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(epsilon * s / (2 * sensitivity)) for s in scores]
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)
    # Choose a candidate based on the probabilities
    private_query_inx = np.random.choice(list(range(0,len(scores))), 1, p=probabilities)[0]
    
    max_utility = max(scores)
    private_query_utility = scores[private_query_inx]
    if max_utility != 0: 
        print('QUERY: ',real_query)
        print('NUM OF CANDIDATES THAT CONTAIN THE QUERY: %d' % sum(u!=0 for u in scores))
        print('MAX UTILITY: '+str(max_utility))
        print('MAX UTILITY QUERY: '+str(cands[scores.index(max_utility)]))
        print('PRIVATE QUERY UTILITY: '+str(private_query_utility))
        print('PRIVATE QUERY: ',cands[private_query_inx])
    else:
        return cands[private_query_inx], -1
    if private_query_utility == 0:
        return cands[private_query_inx], 1
    else:
        return cands[private_query_inx], 0

# candidate preparation
def preprocess(query_type,max_preds_num,dataset_name):
    if dataset_name=='imdb':
        df = create_imdb()
        space_cols = [['startYear']]
    if dataset_name=='ebay':
        df = create_ebay()
        #space_cols = [['Manufacturer'],['Title'],
        #              ['Carrier','Color Category'],['Carrier','Manufacturer']]
        space_cols = [['Manufacturer'],['Carrier'],['Color Category'],['Title'],
                      ['Carrier','Color Category'],['Carrier','Manufacturer'],['Color Category','Manufacturer']]
    all_res = {}
    cands = []
    candstrs = []
    for pred_cols in space_cols: 
        # candidates with multiple column values as predicates
        cs = create_candidates_columns(df,pred_cols,max_preds_num)
        cands.extend(cs)
        # building the string of cands
        for c in cs:
            pstr = [str(z)+' = '+str(w) for (z,w) in zip(pred_cols,c)]
            predstr = query_type.join(pstr) 
            candstrs.append(predstr)
        #
        # compute the results of candidates
        all_res.update(get_all_cand_results(df,pred_cols,cs,query_type).copy())
    print('TOTAL NUM OF CANDIDATES: %d' % (len(cands)))
    
    with open(dataset_name+'_candidates.json', 'w') as f:
        json.dump(candstrs, f)
    with open(dataset_name+'_queryresults.json', 'wb') as f:
        pickle.dump(all_res, f)
    return df,all_res,candstrs


#def run_queries(dataset_name,query_type,query_num,all_res,cands,query_column):
def run_queries(dataset_name,query_type,query_num,query_column):
    # read dataset and load preprocessed data
    if dataset_name=='imdb':
        df = create_imdb()
    if dataset_name=='ebay':
        df = create_ebay()
    with open(dataset_name+'_candidates.json', 'r') as f:
        cands = json.load(f)
    with open(dataset_name+'_queryresults.json', 'rb') as f:
        all_res = pickle.load(f)
    # read queries    
    qs = read_queries(dataset_name)
    print('NUM OF QUERIES: ' + str(len(qs)))
    print('---------------------------')
    #
    epsilon = 1 
    sensitivity = 1
    # compute candidate results
    q_res = get_all_cand_results(df,[query_column],qs,query_type)
    all_res.update(q_res)
    zero_max_utility = 0
    zero_private_utility = 0
    # make queries string
    qstrs = [query_column+' = '+q[0] for q in qs]
    fns = [utility1,utility2,utility3]
    zero_max_utilities = [0,0,0]
    zero_private_utilities = [0,0,0]
    
    for i in range(len(qs)): 
        q = qstrs[i]
        cands_query = [c for c in cands if qs[i][0] not in c]
        #print('NUM OF CANDIDATES FOR THIS QUERY: %d' % (len(cands_query)))
        print('---------------------------')
        for i in range(len(fns)):
            utility_fn = fns[i]
            print('UTILIT FUNC %d' %(i))
            pq, pqu = private_query(q,epsilon,cands_query,all_res,sensitivity,utility_fn)
            
            if pqu == -1: 
                #zero_max_utility += 1
                zero_max_utilities[i] +=1
            else:
                #zero_private_utility += pqu
                zero_private_utilities[i] += pqu
            print('---------------------------')
                
    for i in range(len(fns)):  
        print('UTILITY FUNC %d: %d OUT OF %d QUERIES DO NOT HAVE CONTAINING QUERIES'% (i, zero_max_utilities[i],len(qs)))
        print('UTILITY FUNC %d: %d OUT OF %d QUERIES HAVE CONTAINING QUERIES AND ZERO UTILITY PRIVATE QUERIES'% (i, zero_private_utilities[i],len(qs)))
            

    #print('%d OUT OF %d QUERIES DO NOT HAVE CONTAINING QUERIES'% (zero_max_utility,len(qs)))
    #print('%d OUT OF %d QUERIES HAVE CONTAINING QUERIES AND ZERO UTILITY PRIVATE QUERIES'% (zero_private_utility,len(qs)))
            

query_column='Model Name'
keyword='samsung galaxy'
##keyword='samsung galaxy note20 5g'
##keyword='apple'
query_type = ' OR '
##query_type = ' AND '
max_preds_num = 500#None
query_num = 50
dataset_name='ebay'

#create_queries(dataset_name,query_num,query_column,keyword)

#df,all_res,candstrs = preprocess(query_type,max_preds_num,dataset_name)
run_queries(dataset_name,query_type,query_num,query_column)

#df,all_res,cands = preprocess(dataset='imdb')
#run_queries(df,all_res,cands,keyword='Drama,Short',query_column='genres',gencont=True)




NUM OF QUERIES: 50
---------------------------
---------------------------
UTILIT FUNC 0
QUERY:  Model Name = carte mere samsung galaxy a5 2016 a510f motherboard for sale online
NUM OF CANDIDATES THAT CONTAIN THE QUERY: 3
MAX UTILITY: 4.1257529499133594e-05
MAX UTILITY QUERY: Carrier = nan OR Color Category = nan
PRIVATE QUERY UTILITY: 0.0
PRIVATE QUERY:  Manufacturer = amplifier bluetooth motorcycle mp3 stereo speaker audio system radio usb tf
---------------------------
UTILIT FUNC 1
QUERY:  Model Name = carte mere samsung galaxy a5 2016 a510f motherboard for sale online
NUM OF CANDIDATES THAT CONTAIN THE QUERY: 3
MAX UTILITY: 4.125582738561822e-05
MAX UTILITY QUERY: Carrier = nan OR Color Category = nan
PRIVATE QUERY UTILITY: 0.0
PRIVATE QUERY:  Manufacturer = 3x r39 e14 reflector spotlight bulb indoor outdoor lighting replace bulb 25w
---------------------------
UTILIT FUNC 2
QUERY:  Model Name = carte mere samsung galaxy a5 2016 a510f motherboard for sale online
NUM OF CANDIDATES 